In [ ]:
import pandas as pd

import numpy as np
df = pd.read_csv("preprocessed_enron.csv")
df['body'] 

for i, msg in enumerate(df['body']):
    # print(i, msg)

    if msg is np.nan:
        df = df.drop(i)
for i, msg in enumerate(df['body']):
    if msg is np.nan:
        print(i, msg)

df = df.reset_index(drop=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
documents = df['body'].tolist()

# for i, msg in enumerate(df['body']):
#     # print(i, msg)
#     if msg is np.nan:
#         print(i, msg)

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(documents) # keyword frequency list
tf_vectorizer = CountVectorizer()
tf_vectors = tf_vectorizer.fit_transform(documents)         # word frequency list
# del tf_vectorizer
# del tfidf_vectorizer# データ分割r


In [ ]:
tfidf_vectors.shape

In [ ]:
# 作成された辞書を作る　:トレインデータ・テストデータ両方に対応
words=tfidf_vectorizer.get_feature_names_out()
print(words[5020:5025])

In [ ]:
tfidf_mat = tfidf_vectors.toarray() # dead every time
del tfidf_vectors
tf_mat = tf_vectors.toarray()
del tf_vectors
df['tf'] = tf_mat.tolist()
df['tfidf'] = tfidf_mat.tolist()

# データ分割

In [ ]:
from sklearn.model_selection import train_test_split
X_tfidf_train, X_tfidf_test, Y_tfidf_train, Y_tfidf_test = train_test_split(df['tfidf'],df['author'],test_size=0.2,shuffle=True)
X_tf_train, X_tf_test, Y_tf_train, Y_tf_test = train_test_split(df['tf'],df['author'],test_size=0.2,shuffle=True)

In [ ]:
# How many author?
authors = set(Y_tfidf_test)
authors_list = [author for author in authors]

# Feature_vectors の作成
size: 著者の数

Train データから作る

In [ ]:
# df_X = pd.DataFrame(X_tf_train.values.tolist())
# df_Y = pd.DataFrame(Y_tf_train.values.tolist())
df_concat = pd.concat((X_tfidf_train, Y_tfidf_train.rename('author')), axis=1)

reference_vectors = {}
for author in authors:
    
    df_author = df_concat.groupby('author').get_group(author)

    matrix = []
    for row in df_author['tfidf']:
        matrix.append(row)

    np_matrix = np.array(matrix)

    mean_vector = np_matrix.mean(axis=0)
    reference_vectors[author] = mean_vector.tolist()

In [ ]:
authors

# Problem

Tf-idf でも　the や　to などの一般的な単語が強調されてしまった。emails のデータセットの場合だと、全ての文章に含まれているわけではないので抑制作用が弱くなる

# Solution

nltk の　stop_words から commonly used words を　取得し、それらを省いた中でのtop20 を出す

In [ ]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [85]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [93]:
print(stop_words)
len(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

179

In [2]:
# start からend までのwordの配列を返す
def extract_features_words(freq_vector, words,stop_words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)  

        if max_word not in stop_words:
            if count>= start:
                result.append(max_word)
            count+=1
        # count += 1
        
    
    return result

In [1]:
# start からend までのword IDの配列を返す
def extract_features(freq_vector, words,stop_words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)  

        if max_word not in stop_words:
            if count>= start:
                result.append(max_index)
            count+=1
        # count += 1
        
    
    return result

In [3]:
vec = reference_vectors['v.weldon@enron.com']
top_words = extract_features_words(vec, words=words, stop_words=stop_words,  start=0, end=150)
print(top_words)


NameError: name 'reference_vectors' is not defined

# ReferenceVectors 完成

## next

# predict() 関数の作成

In [ ]:
def predict(questioned_vector, reference_vectors):
    suspected = [author for author in authors]

    comparedSize = 20
    while(len(suspected) > 1):
        
        
    

    